In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import plotly.express as px
import re
# import nltk 
# nltk.download('omw-1.4')
# from nltk.corpus import stopwords
# from nltk.stem.wordnet import WordNetLemmatizer



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/all-products-available-on-sephora-website/sephora_website_dataset.csv


In [2]:
import nltk

In [3]:
beauty_products = pd.read_csv('../input/all-products-available-on-sephora-website/sephora_website_dataset.csv')
beauty_products.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
0,2218774,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,5 x 0.16oz/5mL,4.0,4,3002,66.0,75.0,...,True,online only,no options,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...,1,0,0,0
1,2044816,Acqua Di Parma,Cologne,Colonia,0.7 oz/ 20 mL,4.5,76,2700,66.0,66.0,...,True,online only,- 0.7 oz/ 20 mL Spray - 1.7 oz/ 50 mL Eau d...,An elegant timeless scent filled with a fresh-...,no instructions,unknown,1,0,0,0
2,1417567,Acqua Di Parma,Perfume,Arancia di Capri,5 oz/ 148 mL,4.5,26,2600,180.0,180.0,...,True,online only,- 1oz/30mL Eau de Toilette - 2.5 oz/ 74 mL E...,Fragrance Family: Fresh Scent Type: Fresh Citr...,no instructions,Alcohol Denat.- Water- Fragrance- Limonene- Li...,1,0,0,0
3,1417617,Acqua Di Parma,Perfume,Mirto di Panarea,2.5 oz/ 74 mL,4.5,23,2900,120.0,120.0,...,True,online only,- 1 oz/ 30 mL Eau de Toilette Spray - 2.5 oz/...,Panarea near Sicily is an an island suspended ...,no instructions,unknown,1,0,0,0
4,2218766,Acqua Di Parma,Fragrance,Colonia Miniature Set,5 x 0.16oz/5mL,3.5,2,943,72.0,80.0,...,True,online only,no options,The Colonia Miniature Set comes in an iconic A...,Suggested Usage:-Fragrance is intensified by t...,Colonia: Alcohol Denat.- Water- Fragrance- Lim...,1,0,0,0


## There are a lot of categories so I want to group them into Skin care, hair car, makeup and body.

In [4]:
hair_care=['Hair Styling Products','Shampoo',
           'Scalp & Hair Treatments','Hair Masks',
           'Hair','Hair Accessories,Hair Oil',
           'Hair Spray','Hair Straighteners & Flat Irons',
           'Hair Primers,Hair Brushes & Combs,Hair Dryers' ,
            'Hair Products','Hair Removal','Shampoo & Conditioner',
           'Hair Removal & Shaving','Hair Thinning & Hair Loss',
           'High Tech Tools','Hair Styling & Treatments','Curls & Coils']

In [5]:
makeup_products=['Eye Palletts','Lipstick','Face Brushes','Mascara','Eyeliner',
                 'Setting Spray & Powder','Makeup','Concealer',
                 'Eyebrow','Eye Brushes','Lip Gloss','Blush','Bronzer','Makeup Remover',
                'Powder Brush','Lid Shadow Brush','Lip Brushes','Blotting Papers',
                 'Eyelash Curlers','BB & CC Cream']

In [6]:
skin_care=["Face","Facial","Moisturizer","aging","Toners","Sheet","Acne",
           "Moisturizers","Face Wash & Cleansers","Face Serums","Eye Creams & Treatments",
          "Lotions & Oils","Lip Balm & Treatment","Face Sunscreen",
           "Exfoliators","Scrub & Exfoliants","Lip Sunscreen","Skincare Sets",
           "Blemish & Acne Treatments","Sheet Masks","Moisturizer & Treatments",
          "For Face"]

In [7]:
body=['Perfume','Rollerballs & Travel Size','Perfume Gift Sets','Beauty Supplements',
      'Body Lotions & Body Oils','Mists & Essences','Bath & Shower','Body Wash & Shower Gel',
      'Body Sprays & Deodorant','Tweezers & Eyebrow Tools','Self Tanners','Shaving']

In [8]:
pd.set_option('display.max_rows', None)
beauty_products.category.value_counts(dropna=False)

Perfume                            665
Moisturizers                       451
Face Serums                        384
Value & Gift Sets                  378
Face Wash & Cleansers              247
Face Masks                         230
Rollerballs & Travel Size          228
Hair Styling Products              224
Eye Palettes                       202
Lipstick                           191
Eye Creams & Treatments            191
Shampoo                            186
Face Brushes                       183
Highlighter                        169
Foundation                         163
Cologne                            158
Mini Size                          154
Mascara                            154
Conditioner                        147
Candles & Home Scents              144
Face Primer                        144
Perfume Gift Sets                  136
Eyeliner                           126
Beauty Supplements                 118
Setting Spray & Powder             116
Body Lotions & Body Oils 

In [9]:
missing_values_count = beauty_products.isnull().sum()
missing_values_count

#No missing variables, awesome!

id                        0
brand                     0
category                  0
name                      0
size                      0
rating                    0
number_of_reviews         0
love                      0
price                     0
value_price               0
URL                       0
MarketingFlags            0
MarketingFlags_content    0
options                   0
details                   0
how_to_use                0
ingredients               0
online_only               0
exclusive                 0
limited_edition           0
limited_time_offer        0
dtype: int64

Is There a correlation btween price and loved?

In [10]:
hair_products=pd.DataFrame(beauty_products.loc[beauty_products.category.isin(hair_care)].groupby('brand')['price'].mean()).reset_index().sort_values(by='price', ascending=False).reset_index().drop('index',axis=1)
hair_products.head(11)

,brand,price
0,dyson,524.000000
1,ghd,207.750000
2,Bio Ionic,178.000000
3,T3,166.333333
4,Google,129.000000
5,StackedSkincare,75.000000
6,shu uemura,49.516129
7,Rahua,46.843750
8,Rossano Ferretti Parma,45.500000
9,Reverie,45.000000


In [11]:
hair_products=pd.DataFrame(beauty_products.loc[beauty_products.category.isin(hair_care)].groupby('brand')['price'].mean()).reset_index().sort_values(by='price', ascending=False).reset_index().drop('index',axis=1)
hair_products.head(10)
sliced_df=hair_products.iloc[0:10]
print("Most expensive hair care products")
px.pie(sliced_df,names=sliced_df.brand,values=sliced_df.price)


Most expensive hair care products


In [12]:
makeup_products=pd.DataFrame(beauty_products.loc[beauty_products.category.isin(makeup_products)].groupby('brand')['price'].mean()).reset_index().sort_values(by='price', ascending=False).reset_index().drop('index',axis=1)
makeup_products.head(10)

,brand,price
0,LANCER Skincare,150.000000
1,La Mer,86.000000
2,Peter Thomas Roth,85.000000
3,Christian Louboutin,81.875000
4,Koh Gen Do,64.666667
5,surratt beauty,62.578947
6,LASHFOOD,62.000000
7,AMOREPACIFIC,60.000000
8,TOM FORD,56.321429
9,Artis,55.285714


In [13]:
skincare=pd.DataFrame(beauty_products.loc[beauty_products.category.isin(skin_care)].groupby('brand')['price'].mean()).reset_index().sort_values(by='price', ascending=False).reset_index().drop('index',axis=1)
skincare.head(10)

,brand,price
0,SK-II,193.800000
1,Dr. Barbara Sturm,189.642857
2,La Mer,174.545455
3,Tata Harper,159.888889
4,Guerlain,158.392857
5,StackedSkincare,143.333333
6,HERMÈS,143.000000
7,Omorovicza,131.000000
8,AMOREPACIFIC,126.866667
9,goop,125.000000


In [14]:
body=pd.DataFrame(beauty_products.loc[beauty_products.category.isin(body)].groupby('brand')['love'].mean()).reset_index().sort_values(by='love', ascending=False).reset_index().drop('index',axis=1)
body.head(10)

,brand,love
0,Tatcha,120450.000000
1,Glow Recipe,67200.000000
2,FENTY BEAUTY by Rihanna,58300.000000
3,Caudalie,47538.800000
4,tarte,45400.000000
5,Sol de Janeiro,37276.555556
6,SK-II,25200.000000
7,KVD Vegan Beauty,19950.000000
8,PATRICK TA,18650.000000
9,Omorovicza,17500.000000


In [15]:
available_brands=beauty_products.brand.value_counts()
print("availabe brands at Sephora and amount")
px.bar(data_frame=available_brands)

availabe brands at Sephora and amount


In [16]:
df=pd.DataFrame(beauty_products.groupby('brand')['love'].mean()).reset_index().sort_values(by='love', ascending=False).reset_index().drop('index',axis=1)
df.head(10)
sliced_df=df.iloc[0:10]
print("Top 10 most loved Brands at Sephora")
px.pie(sliced_df,names=sliced_df.brand,values=sliced_df.love)


Top 10 most loved Brands at Sephora


In [17]:
sliced_df=df.iloc[0:10]
print("Top 10 most loved Brands at Sephora")
px.pie(sliced_df,names=sliced_df.brand,values=sliced_df.love)

Top 10 most loved Brands at Sephora


Indgredients

In [18]:
beauty_products['details'][:2]

0    This enchanting set comes in a specially handc...
1    An elegant timeless scent filled with a fresh-...
Name: details, dtype: object

In [19]:
df_skin = beauty_products[beauty_products['category'].isin(makeup_products)].reset_index()